In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py
h5py._errors.unsilence_errors()
from mpl_toolkits.axes_grid1 import make_axes_locatable

# SUGGESTION: create all folders for storing results
if not os.path.exists('./vis'):
    os.mkdir('./vis')

if not os.path.exists('./vis_results'):
    os.mkdir('./vis_results')

if not os.path.exists('./model256_weights'):
    os.mkdir('./model256_weights')


In [21]:
#import other module

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import sys
sys.path.append('/kaggle/input/helpfunction/')



/kaggle/input/track2dataset/256modelruns/Pe1_K1_0_0.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_1_1.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_3_3.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_2_2.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_3_0.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_0_3.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_1_2.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_3_1.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_1_3.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_0_1.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_2_0.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_0_2.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_2_1.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_1_0.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_3_2.hdf5
/kaggle/input/track2dataset/256modelruns/Pe1_K1_2_3.hdf5
/kaggle/input/helpfunction/dataset.py
/kaggle/input/helpfunction/pre_processing.py


In [22]:
!pip install -U segmentation_models_pytorch
!pip install lightning



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


    Import Lightning: Import the necessary modules from PyTorch Lightning

In [23]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split


import lightning as L

import albumentations as albu

import segmentation_models_pytorch as smp
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm

import torch.nn.functional as F
import pandas as pd

from itertools import product
import h5py


from torch.utils.data import RandomSampler

    Define LightningModule: Create a LightningModule class that inherits from pl.LightningModule. This class will contain your model architecture and training logic.



In [24]:
class MyLightningModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        
        self.model = smp.Unet(
            encoder_name='resnet34',
            encoder_weights=None,
            in_channels=4,
            classes=1,
            activation='sigmoid'
        )
        self.l2_loss = torch.nn.MSELoss()
        self.l1_loss = torch.nn.L1Loss()
        
        #save all hyperparameters
        self.save_hyperparameters()
        
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        imgs, masks = batch
        preds = self.model(imgs).squeeze()
        loss = self.l2_loss(preds, masks)
        
        self.log('train_loss', loss, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        imgs, masks = batch
        preds = self.model(imgs).squeeze()
        val_loss = self.l2_loss(preds, masks)
        
        self.log('val_loss', val_loss, prog_bar=True)
        
        return val_loss
    
    def test_step(self, batch, batch_idx):
        
        imgs, masks = batch
        preds = self.model(imgs).squeeze()
        test_loss = self.l2_loss(preds, masks)
        
        self.log("test_loss", test_loss, prog_bar=True)
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam([ dict(params=self.model.parameters(), lr=5e-4),])
        
        return optimizer




    Define LightningDataModule: If you're using custom data loaders, create a LightningDataModule class that inherits from pl.LightningDataModule. This class will contain your data loading logic.

In [25]:

from dataset import MyDataset

    
class MyDataModule(L.LightningDataModule):
    def __init__(self, augmentation=None, preprocessing=None, batch_size=32):
        super().__init__()
        self.batch_size = batch_size
        self.augmentation = augmentation
        self.preprocessing = preprocessing
        self.n_training_samples = 10
        self.n_valid_samples = 2
        self.n_test_samples = 4

        
    def setup(self, stage=None):

        #get the file names
        permutations = list(product(range(4), repeat=2))
        file_list = []
        properties_list = []
        for idx1, idx2 in permutations:
            file_name = f'/kaggle/input/track2dataset/256modelruns/Pe1_K1_{idx1}_{idx2}.hdf5'
            file_list.append(file_name)


        # # set breakpoint
        # import pdb
        #pdb.set_trace()
        #self.example_dataset = MyDataset(file_list[:3],self.augmentation[0], self.preprocessing)
        
        self.train_dataset = MyDataset(file_list[:self.n_training_samples],self.augmentation[0], self.preprocessing)
        
        self.val_dataset = MyDataset(file_list[self.n_training_samples : self.n_training_samples+self.n_valid_samples],self.augmentation[1], self.preprocessing)
        
        self.test_dataset = MyDataset(file_list[-self.n_test_samples :], self.augmentation[2], self.preprocessing)
     
             
    def train_dataloader(self):
        train_sampler = RandomSampler(self.train_dataset, replacement=True, num_samples=10000) 
        return DataLoader(self.train_dataset, batch_size=self.batch_size, sampler=train_sampler, num_workers=2, drop_last=True, persistent_workers=True) # 

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=2, shuffle=False, drop_last=True, persistent_workers=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=2,persistent_workers=True)


    Training Loop with Trainer: Create a pl.Trainer object and use it to train your LightningModule.

* from commandline, type tensorboard --logdir=lightning_logs/

If you’re using a notebook environment such as colab or kaggle or jupyter, launch Tensorboard with this command

%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

In [26]:


def get_training_augmentation():
    train_transform = [
        albu.Resize(256, 256),  # not needed
        # albu.HorizontalFlip(p=0.5),
        # albu.VerticalFlip(p=0.5),
    ]
    return albu.Compose(train_transform)

def get_validation_augmentation():
    """Resize to make image shape divisible by 32"""
    test_transform = [
        albu.Resize(256, 256),  
    ]
    return albu.Compose(test_transform)

In [ ]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelSummary
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.callbacks import DeviceStatsMonitor
from lightning.pytorch.profilers import AdvancedProfiler

from pre_processing import get_preprocessing

from lightning.pytorch.loggers import TensorBoardLogger

#logger = TensorBoardLogger("tb_logs", name="my_model")

augumentations=[get_training_augmentation(), get_validation_augmentation(), get_validation_augmentation()]
# if __name__ == "__main__":

model = MyLightningModel()

data_module = MyDataModule(augmentation=augumentations, preprocessing=get_preprocessing())



# profiler = AdvancedProfiler(dirpath=".", filename="lightning_logs/perf_logs")
# 
#profiler=profiler, default_root_dir='/Users/captainjack/Desktop/CO2_Storage_Jack/'

#consider trying this https://lightning.ai/docs/pytorch/stable/common/precision_intermediate.html

#fast_dev_run=True,
#logger=logger

trainer = L.Trainer(max_epochs=20, profiler="advanced",\
                     callbacks=[DeviceStatsMonitor(), ModelSummary(max_depth=-1), EarlyStopping(monitor="val_loss", min_delta=0.01, patience=3, verbose=False, mode="max")])

#VisualizationCallback(data_module)
#TQDMProgressBar(refresh_rate=10)

# check validation before large training step
#num_sanity_val_steps=2, 


# 
# trainer = Trainer(callbacks=[DeviceStatsMonitor()])

trainer.fit(model, data_module)


# test the model 
trainer.test(model, data_module) 


INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: 
    | Name                                        | Type             | Params
-----------------------------------------------------------------------------------
0   | model                                       | Unet             | 24.4 M
1   | model.encoder                               | ResNetEncoder    | 21.3 M
2   | model.encoder.conv1                         | Conv2d           | 12.5 K
3   | model.encoder.bn1                           | BatchNorm2d      | 128   
4   | model.encoder.relu                          | ReLU             | 0     
5   | model.encoder.maxpool                       | MaxPool2d        | 0     
6   | mod

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
